### Selected Ngram Repeat With Youtube Link

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from nltk import ngrams
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Intersect"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0 native word start index
word_end = 1000  # 28 native word end index

In [4]:
def df_col_value_join_comma(df, df_columns_list):
    '''
    df_col_value_join_comma(df_test, ["video_id","start_time","end_time"])\n
    function used for selected column value join with comma in one row
    '''
    column_value_list = []
    for column in df_columns_list:
        list_var = df[f"{column}"].to_list()
        list_var_string = [str(x) for x in list_var] 
        list_var_join = ",".join(list_var_string)
        column_value_list.append(list_var_join)

    df_result = pd.DataFrame([[column_value_list[0],column_value_list[1],column_value_list[2]]], columns=df_columns_list)

    return df_result

In [5]:
#data_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
#Deployment3/Data/Manuel Select"

In [6]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment3/Result/4-Selected Ngram Repeat With Youtube Link Deploy Result"

Path(path).mkdir(parents=True, exist_ok=True)

#### Ngram Link Analysis

##### Link Count

In [43]:
# file extention
file_ext = "Twogram"

In [44]:
path_folder = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment3/Result/Ngram Youtube Link Manuel"
file = f"{lang_folder.capitalize()}_{file_ext}_With_{word_end}_Word_Youtube_Link_Manuel_12.xlsx"   ########*****
#sheet = "Sheet1"  # Sheet1

In [45]:
#df_link_selected_manuel = pd.read_excel(f"{path_folder}/{file}", sheet_name=f"{sheet}")
df_link_selected_manuel = pd.read_excel(f"{path_folder}/{file}")
df_link_selected_manuel

,video_id,start_time,end_time,text
0,NtoRI5nlwKw,1073,1075,duyuyor musun
1,CwuGepQDBrM,153,155,güzel bir
2,UQDUQnVUtq4,942,944,güzel bir
3,dRCz3FhBk08,97,99,güzel bir
4,7XVwNBlTIWs,13574,13576,güzel bir
...,...,...,...,...
973,y7o33BX7ec0,428,430,yardım etmek
974,0qGrmkm6r44,520,522,yardım etmek
975,Chvvzy8loOE,1369,1371,yardım etmek
976,0Hyrz58zvcA,1400,1402,yardım etmek


In [46]:
# disabled because of youtube tools
#df_link_selected_manuel["video_url"] = "https://www.youtube.com/watch?v="+df_link_selected_manuel['video_id'].map(str)+"&t="+df_link_selected_manuel['start_time'].map(str)+"s"
#df_link_selected_manuel

In [47]:
df_count = df_link_selected_manuel.groupby(["text"])[["text"]].count()
df_count.rename(columns={"text":"count"}, inplace=True)
df_count.sort_values(by="count", ascending=False, inplace=True)
df_count.reset_index(inplace=True)
df_count

,text,count
0,ne yapabilirim,14
1,ne olduğunu,10
2,öyle bir,10
3,kimse yok,9
4,ne tür,9
...,...,...
173,yok mu,3
174,şey yok,3
175,beni duyuyor,2
176,neydi böyle,1


In [48]:
df_count = df_count[df_count["count"] > 2]

In [49]:
len(df_count)

175

In [50]:
df_count_merge = pd.merge(df_link_selected_manuel, df_count, how="inner",on= "text")
df_count_merge.sort_values(by=["count","text"], ascending=False, inplace=True)
df_count_merge.reset_index(drop=True, inplace=True)
df_count_merge

,video_id,start_time,end_time,text,count
0,Jfh5yVT4OPk,418,420,ne yapabilirim,14
1,T5krOQidV5s,714,716,ne yapabilirim,14
2,vGkcBnwz8UQ,1,3,ne yapabilirim,14
3,c90800j_ED4,1066,1068,ne yapabilirim,14
4,MaSTWrRUUfY,843,845,ne yapabilirim,14
...,...,...,...,...,...
969,m3Clq7uulPg,4093,4095,bir bak,3
970,GQgS2ruEJAs,5182,5184,bir bak,3
971,n2t0ueWtUNI,54,56,başka bir,3
972,N0a3f_R2BfQ,790,792,başka bir,3


In [38]:
df_count.to_excel(f"{path_folder}/{file_ext} Video List.xlsx", index=False)

In [39]:
df_count_merge.to_excel(f"{path_folder}/{lang_folder.capitalize()}_{file_ext}_With_{word_end}_Word_Youtube_Link_Manuel_Count.xlsx", index=False)   ########*****

##### Link Repeat

In [51]:
# ******  
path_folder_file = f"{path_folder}/{lang_folder.capitalize()}_{file_ext}_With_{word_end}_Word_Youtube_Link_Manuel_Count_Select.xlsx"   # from youtube tool

In [52]:
#df_word_group_time_loc = pd.read_excel(f"{path_folder_file2}", sheet_name=f"{sheet2}")
df_word_group_time_loc = pd.read_excel(f"{path_folder_file}")
#df_word_group_time_loc = pd.read_excel(f"Twogram Threegram Youtube Link Selected_Manuel.xlsx")   
df_word_group_time_loc

,video_id,start_time,end_time,text
0,T5krOQidV5s,714,716,ne yapabilirim
1,c90800j_ED4,1066,1068,ne yapabilirim
2,Ts4gUJI0Iks,986,988,ne yapabilirim
3,Ssg7vpw8Zb4,427,429,ne yapabilirim
4,M0SXeJ45hww,371,373,ne yapabilirim
...,...,...,...,...
907,m3Clq7uulPg,4093,4095,bir bak
908,GQgS2ruEJAs,5182,5184,bir bak
909,n2t0ueWtUNI,54,56,başka bir
910,N0a3f_R2BfQ,790,792,başka bir


In [53]:
df_count2 = df_word_group_time_loc.groupby(["text"])[["text"]].count()
df_count2.rename(columns={"text":"count"}, inplace=True)
df_count2.sort_values(by="count", ascending=False, inplace=True)
df_count2.reset_index(inplace=True)
df_count2

,text,count
0,ne demek,7
1,kadar çok,7
2,tam olarak,7
3,var biliyor,7
4,iyi olacak,7
...,...,...
170,yok mu,3
171,evet ben,3
172,kontrol altında,3
173,söyledi mi,3


In [54]:
df_word_group_time_loc = pd.merge(df_word_group_time_loc, df_count2, how="left", on="text")
df_word_group_time_loc.drop_duplicates(inplace=True)
df_word_group_time_loc

,video_id,start_time,end_time,text,count
0,T5krOQidV5s,714,716,ne yapabilirim,7
1,c90800j_ED4,1066,1068,ne yapabilirim,7
2,Ts4gUJI0Iks,986,988,ne yapabilirim,7
3,Ssg7vpw8Zb4,427,429,ne yapabilirim,7
4,M0SXeJ45hww,371,373,ne yapabilirim,7
...,...,...,...,...,...
907,m3Clq7uulPg,4093,4095,bir bak,3
908,GQgS2ruEJAs,5182,5184,bir bak,3
909,n2t0ueWtUNI,54,56,başka bir,3
910,N0a3f_R2BfQ,790,792,başka bir,3


In [55]:
search_list = df_word_group_time_loc.loc[:,"text"].unique()
len(search_list)

175

In [56]:
# other option 
# mUf7VNqChac =>  black screen
# 0_CDMstFg7M => 10sn
# bj1JRuyYeco => 20sn
# cElhIDdGz7M => screensaver
default_video_id = "Q-8I-uMUMYA"
#df_link_default = pd.DataFrame(data=[["repeat",5,7,"repeat_again",f"{default_video_id}",f"https://www.youtube.com/watch?v={default_video_id}&t=0s"]], columns=["text","start_time","end_time","sentence","video_id","video_url"])
df_link_default = pd.DataFrame(data=[["repeat",5,7,"repeat_again",f"{default_video_id}"]], columns=["text","start_time","end_time","sentence","video_id"])
df_link_default

,text,start_time,end_time,sentence,video_id
0,repeat,5,7,repeat_again,Q-8I-uMUMYA


In [48]:
#(df_result_repeat["end_time"].sum()-df_result_repeat["start_time"].sum())/60

In [49]:
# Each word group result convert to join result in one row
df_all_join_result = pd.DataFrame()
seq_num = 0  # option
for word_group in search_list:
    df_result_repeat = pd.DataFrame()
    # for repeat 1
    df_word_group_search_repeat1 = df_word_group_time_loc[df_word_group_time_loc["text"] == word_group]
    # for repeat 2
    try:
        df_word_group_search_repeat2 = df_word_group_time_loc[df_word_group_time_loc["text"] == word_group].sample(1)
    except:
        df_word_group_search_repeat2 = df_word_group_time_loc[df_word_group_time_loc["text"] == word_group].tail(1)
    df_word_group_search_repeat2.reset_index(drop=True, inplace=True)   
    # repeat block
    for i in range(len(df_word_group_search_repeat1)):        
        df_result_repeat = pd.concat([df_result_repeat,df_word_group_search_repeat1.iloc[[i,]]], axis=0)

    try:
        df_link_default_var = df_link_default
        word_time_diff_var = df_word_group_search_repeat2.loc[0 ,"end_time"] - df_word_group_search_repeat2.loc[0 ,"start_time"]
        if word_time_diff_var < 4:
            word_time_diff_var2 = 3
        else:
            word_time_diff_var2 = word_time_diff_var+1.0

        df_link_default_var.loc[0,"end_time"] = df_link_default_var.loc[0,"start_time"] + word_time_diff_var2
    except:
        pass

    for j in range(3):
        df_result_repeat = pd.concat([df_result_repeat,df_word_group_search_repeat2], axis=0)
        df_result_repeat = pd.concat([df_result_repeat,df_link_default_var], axis=0)
        df_result_repeat.reset_index(drop=True, inplace=True)

    df_result_repeat_var = df_col_value_join_comma(df_result_repeat, ["video_id","start_time","end_time"]) 
    df_result_repeat_var.loc[0,["text"]] = f"{word_group}"  # option for search string
    df_result_repeat_var.loc[0,["count"]] = df_result_repeat.loc[0,"count"] # option for count num     
    df_all_join_result = pd.concat([df_all_join_result,df_result_repeat_var], axis=0)

df_all_join_result.reset_index(drop=True, inplace=True)
df_all_join_result["web_format"] = f"[play_lms_videos video_id="+df_all_join_result["video_id"].map(str)+" start_\
time="+df_all_join_result["start_time"].map(str)+" end_time="+df_all_join_result["end_time"].map(str)+"]"
df_all_join_result.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Youtube_Talk_Time_Join_{file_ext}.xlsx", index=False)

In [50]:
df_all_join_result

,video_id,start_time,end_time,text,count,web_format
0,"4O_QCCZSoYQ,o2guE9xdVt8,o0vdJUDd7hQ,RuVVxns8wh...","38,151,397,190,3408,52,610,190,5,190,5,190,5","40,153,399,192,3410,54,612,192,8,192,8,192,8",hiç böyle bir şey,7.0,"[play_lms_videos video_id=4O_QCCZSoYQ,o2guE9xd..."
1,"f57lDadHW7w,nbyZeB6ovkY,HH9H9diryt8,iw7ytmOa5o...","487,183,1474,764,162,338,98,162,5,162,5,162,5","489,185,1476,766,165,340,100,165,8,165,8,165,8",hiç bir şey yok,7.0,"[play_lms_videos video_id=f57lDadHW7w,nbyZeB6o..."
2,"dzv7nR7Fpus,2SS88Z9WIRw,vdrZQOKvsZM,7jLcYDyLcY...","958,772,66,32,266,83,605,958,5,958,5,958,5","960,774,68,34,268,85,607,960,8,960,8,960,8",gereken bir şey var,7.0,"[play_lms_videos video_id=dzv7nR7Fpus,2SS88Z9W..."
3,"js3Lm9YbWpQ,PxkMtDYEmhA,vdrZQOKvsZM,wLVTXAqOcH...","74,214,583,210,1094,879,1228,74,5,74,5,74,5","76,216,585,212,1096,881,1230,76,8,76,8,76,8",bu ne demek oluyor,7.0,"[play_lms_videos video_id=js3Lm9YbWpQ,PxkMtDYE..."
4,"eLY2q3MiaUQ,h93wXNLXmYU,wBjDafmCd_c,gm9X3qR1GR...","32,1153,4366,187,125,3253,693,3253,5,3253,5,32...","34,1155,4368,189,127,3255,695,3255,8,3255,8,32...",bir şey söyleyeyim mi,7.0,"[play_lms_videos video_id=eLY2q3MiaUQ,h93wXNLX..."
...,...,...,...,...,...,...
58,"TLv5OWfkU8U,vhmLncm06Wc,MKX0zLqY7YU,TLv5OWfkU8...","6207,2986,1290,6207,5,6207,5,6207,5","6209,2988,1292,6209,8,6209,8,6209,8",bir şey yok mu,3.0,"[play_lms_videos video_id=TLv5OWfkU8U,vhmLncm0..."
59,"h07c3ds0dGw,MvUL_IHyESU,TO1xscJMX04,h07c3ds0dG...","211,1266,299,211,5,211,5,211,5","213,1268,301,213,8,213,8,213,8",bir şey var mı,3.0,"[play_lms_videos video_id=h07c3ds0dGw,MvUL_IHy..."
60,"HhrBZuxjFxM,mJncKRivnNo,NOZqjtwj26M,mJncKRivnN...","4479,311,61,311,5,311,5,311,5","4481,313,63,313,8,313,8,313,8",bir şey söyledi mi,3.0,"[play_lms_videos video_id=HhrBZuxjFxM,mJncKRiv..."
61,"s4MqASVecXc,YYCPETGxTuE,AevAo6AUCiQ,YYCPETGxTu...","3746,161,113,161,5,161,5,161,5","3748,163,115,163,8,163,8,163,8",bir sorun mu var,3.0,"[play_lms_videos video_id=s4MqASVecXc,YYCPETGx..."


##### Check Missing Ngram

In [57]:
master_path_folder_file = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment3/Data/\
Manuel Select/Revised Ngrams All in One.xlsx"
sheet = "Sheet2"

In [58]:
df_file = pd.read_excel(f"{master_path_folder_file}", sheet_name=f"{sheet}")
#df_file = pd.read_excel(f"twogram.xlsx")
df_file

,fourgram,threegram,twogram
0,bana yardım eder misin,NaN,NaN
1,bana yardım eder misin,NaN,NaN
2,bana yardım eder misin,yardım eder misin,eder misin
3,bana yardım eder misin,yardım eder misin,yardım eder
4,başka bir şey daha,başka bir şey,bir şey
...,...,...,...
495,NaN,çok kötü bir,çok kötü
496,NaN,bir şey olmaz,NaN
497,NaN,şuna bir bak,bir bak
498,NaN,şuna bir bak,NaN


In [59]:
df_ngram_not_in_master_result = pd.DataFrame(set(df_file[f"{file_ext.lower()}"].dropna()).difference(set(df_word_group_time_loc["text"])), columns=[f"{file_ext.lower()}"])
df_ngram_not_in_master_result

,twogram
0,beni duyuyor
1,yaptığını sanıyorsun
2,neydi böyle


In [60]:
df_ngram_not_in_master_result.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_{file_ext}_Not_In_Result.xlsx", index=False)

#### Copy Move And Delete

In [51]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{word_end}_Word_Youtube_Talk_Time_Join_*.xlsx")
output_file

['Turkish_1000_Word_Youtube_Talk_Time_Join_Twogram.xlsx',
 'Turkish_1000_Word_Youtube_Talk_Time_Join_Threegram.xlsx',
 'Turkish_1000_Word_Youtube_Talk_Time_Join_Fourgram.xlsx']

In [52]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [53]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass